# 3.1章　LLMに知識を与える

In [ ]:
!pip install langchain
!pip install langchain-openai

In [ ]:
# colabで実行する場合（本には載せない）　TODO：dotenvを利用
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['SERPAPI_API_KEY'] = userdata.get('SERPAPI_API_KEY')

In [ ]:
# 普通に質問

from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

model = ChatOpenAI(model="gpt-4o-mini")
result = model.invoke([HumanMessage(content="熊童子について教えてください。")])

result.content

In [ ]:
# 知識を前もって与えておく

from langchain_core.prompts import ChatPromptTemplate

# プロンプトテンプレートの作成
message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

model = ChatOpenAI(model="gpt-4o-mini")
chain = prompt | model

question_text = "熊童子について教えてください。"
information_text = """
熊童子はベンケイソウ科コチレドン属の多肉植物です。
葉に丸みや厚みがあり、先端には爪のような突起があることから「熊の手」という愛称で人気を集めています。
花はオレンジ色のベル型の花を咲かせることがあります。
"""

response = chain.invoke({"context": information_text, "question": question_text})
response.content

In [ ]:
# Documentクラスで文書を管理

from langchain_core.documents import Document

document = Document(
        page_content="""
セダムはベンケイソウ科マンネングザ属で、日本にも自生しているポピュラーな多肉植物です。
種類が多くて葉の大きさや形状、カラーバリエーションも豊富なので、組み合わせて寄せ植えにしたり、庭のグランドカバーにしたりして楽しむことができます。
とても丈夫で育てやすく、多肉植物を初めて育てる方にもおすすめです。
""",
        metadata={"source": "succulent-plants-doc"},
    )

document

In [ ]:
!pip install langchain_chroma

In [ ]:
# データベースの作成
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

documents = [
    Document(
        page_content="""
セダムはベンケイソウ科マンネングザ属で、日本にも自生しているポピュラーな多肉植物です。
種類が多くて葉の大きさや形状、カラーバリエーションも豊富なので、組み合わせて寄せ植えにしたり、庭のグランドカバーにしたりして楽しむことができます。
とても丈夫で育てやすく、多肉植物を初めて育てる方にもおすすめです。
""",
        metadata={"source": "succulent-plants-doc"},
    ),
    Document(
        page_content="""
熊童子はベンケイソウ科コチレドン属の多肉植物です。
葉に丸みや厚みがあり、先端には爪のような突起があることから「熊の手」という愛称で人気を集めています。
花はオレンジ色のベル型の花を咲かせることがあります。
""",
        metadata={"source": "succulent-plants-doc"},
    ),
    Document(
        page_content="""
エケベリアはベンケイソウ科エケベリア属の多肉植物で、メキシコなど中南米が原産です。
まるで花びらのように広がる肉厚な葉が特徴で、秋には紅葉も楽しめます。
品種が多く、室内でも気軽に育てられるので、人気のある多肉植物です。
""",
        metadata={"source": "succulent-plants-doc"},
    ),
]

vectorstore = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings(),
)


In [ ]:
vectorstore.similarity_search("熊童子")

In [ ]:
vectorstore.similarity_search_with_score("熊童子")

In [ ]:
# retrieverの作成
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)  # select top result
retriever.invoke("熊童子")

In [ ]:
# RAG

from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])
model = ChatOpenAI(model="gpt-4o-mini")

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | model

result = rag_chain.invoke("セダムについて教えてください。")
result.content